In [ ]:
# Import the packages we need

import pandas as pd
import numpy as np
from datetime import *
import math

In [ ]:
# Import the raw transaction level data for each route in a certain month

df1 = pd.read_csv("01_07_2023.csv",low_memory=False)
df2 = pd.read_csv("02_07_2023.csv",low_memory=False)

df = pd.concat([df1,df2]).reset_index(drop=True)

In [ ]:
# Data Cleaning
df['route_zone'] = df['route_zone'].str.strip()

In [ ]:
rz_db = pd.read_csv('rz_db.csv')

In [ ]:
rz_db['Route Zone'] = rz_db['Route Zone'].str.strip()

In [ ]:
# Combine the data together

df2 = df.merge(rz_db[['Route Zone','Zone Type']], left_on='route_zone',right_on='Route Zone',how='left')

In [ ]:
# Convert the dates into datetime so we can work with them easily in the future

df2['route_date'] = pd.to_datetime(df2['route_date'], format='%d/%m/%y')
df2['route_month'] = df2['route_date'].dt.strftime('%Y-%m')

In [ ]:
# Quick checker for a driver by looking up their driver ID

df_test = df2[df2['driver_id']==1454743]
df_test[ (df_test['delivery_type']=='Forward Delivery') & (df_test['transaction_status']=='Success')]['parcel_size'].value_counts()

In [ ]:
df_test[df_test['Route Zone'].isnull()].iloc[0]['Zone Type']

In [ ]:
# Aggregate the transaction level data into route level

route_id = []
route_date = []
route_month = []
routed_driver_display_name = []
routed_driver_type = []
routed_driver_id = []

parcels_delivered = []
cod_parcels = []
rpu_parcels = []
route_fd_success_rate = []

for i in df2['route_ID'].unique():
  route_id.append(i)
  temp = df2[df2['route_ID']==i]
  route_date.append(temp.iloc[0]['route_date'])
  route_month.append(temp.iloc[0]['route_month'])
  routed_driver_display_name.append(temp.iloc[0]['driver_display_name'])
  routed_driver_type.append(temp.iloc[0]['driver_type'])
  routed_driver_id.append(temp.iloc[0]['driver_id'])

  forward_delivery = temp[temp['delivery_type']=='Forward Delivery']

  success_forward_delivery = forward_delivery[forward_delivery['transaction_status']=='Success']

  parcels_delivered.append(len(success_forward_delivery))

  if len(forward_delivery) == 0:
    route_fd_success_rate.append(0)
  else:
    route_fd_success_rate.append(round(len(success_forward_delivery)/len(forward_delivery),4))

  cod = temp[temp['cod_flag']=='cod']
  success_cod = cod[cod['transaction_status']=='Success']
  cod_parcels.append(len(success_cod))

  rpu = temp[temp['delivery_type']=='Return Pickup']
  success_rpu = rpu[rpu['transaction_status']=='Success']
  rpu_parcels.append(len(success_rpu))

In [ ]:
base = pd.DataFrame({'route_id':route_id, 'route_date':route_date, 'route_month':route_month, 'routed_driver_display_name':routed_driver_display_name, 'routed_driver_type':routed_driver_type, 'routed_driver_id':routed_driver_id,
                     'parcels_delivered':parcels_delivered, 'cod_parcels':cod_parcels, 'rpu_parcels':rpu_parcels, 'route_fd_success_rate':route_fd_success_rate})

In [ ]:
# base[base['routed_driver_display_name']=='H2 - HMB - ONG BENG YEE']

In [ ]:
# Calculate the estimated pay for each route based on zone type, driver tpye, parcel size, cod, rpu, and the route date

estimated_pay = []

for i in df2['route_ID'].unique():
  temp = df2[df2['route_ID']==i].reset_index(drop=True)
  temp = temp[temp['transaction_status']=='Success'].reset_index(drop=True)

  pay = 0

  for j in range(0,len(temp)):
    temp2 = temp.iloc[j]
# ...
  estimated_pay.append(pay)

In [ ]:
base['estimated_pay'] = estimated_pay

In [ ]:
base[base['routed_driver_id']==1454743]['estimated_pay'].sum()